# Initialization

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
import scipy.interpolate as interp
%matplotlib qt5
eps = np.finfo(float).eps

In [2]:
import sys
import time
import json

## Load backprojection library

In [3]:
sys.path.insert(0, "/home/pleroy/DEV/processing/PoSAR-MC/backprojection")

In [4]:
from loadbackprojection import *

In [5]:
libraryFilename = "/home/pleroy/DEV/processing/PoSAR-MC/backprojection/ccpp/libbackprojection/liblibbackprojection.so"

In [6]:
lib = LibBackProjection( libraryFilename )

## Load other tools

In [7]:
sys.path.insert(0, "/home/pleroy/DEV/processing/focalization_python")

In [8]:
import posarutils.process.disp_PoSAR_img as disp
from posarutils.process.disp_PoSAR_img import OPTt
from posarutils.process.filtering import box_filter

## Load parameters specific to the dataset

In [9]:
from rsc.datasetconfig import *

%load_ext autoreload
%autoreload 2

In [10]:
withPlots = 0

# Read parameters from the XML file

In [11]:
from posarutils.other.PosarMCParameters import *

In [12]:
data_date = "2018_06_27_13_05_40"
root_dir = "/home/pleroy/DATA/2018_06_27_LETG/2018_06_27/jde/"
data_dir = root_dir + data_date

In [13]:
params_filename = data_dir + "/" + data_date + "_parameters.xml"
params = PosarMCParameters_v2( params_filename )
Tp = params.configuredTRamp / 1e6
B0 = params.frequencyBand
fs = params.samplingFrequency
c = 3e8

# Load the analytic signal

In [14]:
rampUp = 0
withHanning = 1
nav = 0

In [15]:
firstFile = 0
nbFiles = 321
lastFile = firstFile + nbFiles - 1
firstRamp = (firstFile) * params.rampsPerFile
lastRamp = (lastFile) * params.rampsPerFile
if withHanning:
    if rampUp:
        RD1 = np.load( data_dir + '/RD_files_{}_{}_hanning.npy'.format( firstFile, lastFile ) )
        coupling = np.load( data_dir + '/coupling_RD_files_{}_{}_hanning.npy'.format( firstFile, lastFile ) )
    else:
        RD1 = np.load( data_dir + '/RD_files_{}_{}_rampDown_hanning.npy'.format( firstFile, lastFile ) )
        coupling = np.load( data_dir + '/coupling_RD_files_{}_{}_rampDown_hanning.npy'.format( firstFile, lastFile ) )
else:
    if rampUp:
        RD1 = np.load( data_dir + '/RD_files_{}_{}.npy'.format( firstFile, lastFile ) )
        coupling = np.load( data_dir + '/coupling_RD_files_{}_{}.npy'.format( firstFile, lastFile ) )
    else:
        RD1 = np.load( data_dir + '/RD_files_{}_{}_rampDown.npy'.format( firstFile, lastFile ) )
        coupling = np.load( data_dir + '/coupling_RD_files_{}_{}_rampDown.npy'.format( firstFile, lastFile ) )

# remove coupling from RD1 to build RD2
#RD2 = RD1 - coupling
nbPos = params.rampsPerFile * nbFiles

In [16]:
if withPlots:
    plt.figure()
    plt.plot(np.abs(coupling))
    plt.grid()

In [17]:
if 0:
    plt.figure()
    plt.plot(np.real(RD1[0,:]))
    plt.plot(np.real(RD1[-187,:]))

# Load antenna positions

In [18]:
# load positions for all ramps
if rampUp:
    filename = "rampNumber_timeStamp_xyz_tM_rampUp.npy"
else:
    if nav:
        filename = "rampNumber_timeStamp_xyz_tM_nav.npy"
    else:
        filename = "rampNumber_timeStamp_xyz_tM_gps.npy"
        
xyz = np.load( data_dir + "/" + filename )
xa = xyz[:,2]
ya = xyz[:,3]
za = xyz[:,4]
xa_mean = np.mean(xa)
ya_mean = np.mean(ya)
za_mean = np.mean(za)
print( filename )
print( "xa_mean = {:.2f}, ya_mean = {:.2f}, za_mean = {:.2f}".format( xa_mean, ya_mean, za_mean ) )

rampNumber_timeStamp_xyz_tM_gps.npy
xa_mean = -151.57, ya_mean = -10.34, za_mean = 292.09


In [19]:
print( "RD1.shape = {}, xyz.shape = {}".format( RD1.shape, xyz.shape ) )

RD1.shape = (240750, 3000), xyz.shape = (248250, 5)


In [20]:
if withPlots == 1:
    plt.figure()

    title = data_date + " selection {} to {}".format( firstFile, lastFile )
    
    plt.suptitle( title )

    plt.subplot(221)
    plt.plot(xyz[:, 1], xyz[:, 2], label="x ")
    plt.plot(xyz[firstRamp:lastRamp, 1], xyz[firstRamp:lastRamp, 2], 'orange', label="x selection")
    plt.grid()
    plt.legend()

    plt.subplot(222)
    plt.plot(xyz[:, 1], xyz[:, 3], label="y ")
    plt.plot(xyz[firstRamp:lastRamp, 1], xyz[firstRamp:lastRamp, 3], 'orange', label="y selection")
    plt.grid()
    plt.legend()

    plt.subplot(223)
    plt.plot(xyz[:, 1], xyz[:, 4], label="z ")
    plt.plot(xyz[firstRamp:lastRamp, 1], xyz[firstRamp:lastRamp, 4], 'orange', label="z selection")
    plt.grid()
    plt.legend()

    plt.subplot(224)
    plt.plot(xyz[:, 2], xyz[:, 3], label="xy ")
    plt.plot(xyz[firstRamp:lastRamp, 2], xyz[firstRamp:lastRamp, 3], 'orange', label="xy (selection)")
    ax = plt.gca()
    ax.invert_xaxis()
    ax.invert_yaxis()
    ax.xaxis.tick_top()
    ax.yaxis.tick_right()
    plt.grid()
    plt.legend()
    
    plt.savefig( data_dir + "/" + title + ".png", bbox_inches='tight')

# Scene geometry

## Define geometry

In [21]:
hScene = 0

d_az = 1.
d_rg = 1.
azMin = -500
azMax = 500
rgMin = 0
rgMax = 500

sceneAz = np.arange(azMin, azMax, d_az)
sceneRg = np.arange(rgMin, rgMax, d_rg)
nbAz = sceneAz.size
nbRg = sceneRg.size
    
sceneAz_mean = np.mean(sceneAz)
sceneRg_mean = np.mean(sceneRg)

In [24]:
if 1:
    plt.figure()
    plt.subplot(211)
    plt.plot( sceneAz, np.ones(sceneAz.shape) * sceneRg[0], 'or', label = "baseline" )
    plt.plot( np.ones(sceneRg.shape) * sceneAz[0], sceneRg, 'or', label = "baseline" )
    plt.plot( xa, ya, 'b', label="antenna positions" )
    plt.plot( xyz[firstRamp:lastRamp, 2], xyz[firstRamp:lastRamp, 3], 'limegreen', label="ramps" )
    ax = plt.gca()
    ax.invert_xaxis()
    ax.invert_yaxis()
    plt.grid()
    plt.legend()
    plt.xlabel("azimuth")
    plt.ylabel("range")
    
    plt.subplot(212)
    plt.plot( xa, za, 'b', label="antenna positions" )
    plt.plot( xyz[firstRamp:lastRamp, 2], xyz[firstRamp:lastRamp, 4], 'limegreen', label="ramps" )
    ax = plt.gca()
    ax.invert_xaxis()
    ax.invert_yaxis()
    plt.grid()
    plt.legend()
    plt.xlabel("azimuth")
    plt.ylabel("elevation")

In [25]:
groundRange = 0 # 0 => slant range, 1 => ground range

# Focalization

In [26]:
RD = np.fft.ifftshift(RD1, 1)

In [27]:
sr = RD
Naz = sr.shape[0]
Nf = sr.shape[1]
overSamplingRatio = 10
Nover = overSamplingRatio * Nf
rangeResolution = c / (2 * B0)
r_base = np.arange( Nf ) * rangeResolution
r_over = np.arange( Nover ) * rangeResolution / overSamplingRatio
dr_over = r_over[1] - r_over[0]

print( "Nf = {}, Naz = {}".format( Nf, Naz ) )
print( "range from {:.2f}m to {:.2f}m, resolution = {}m, oversampled = {}m, ".format(
    r_over[0], r_over[-1], rangeResolution, rangeResolution / overSamplingRatio ) )

Nf = 3000, Naz = 240750
range from 0.00m to 2999.90m, resolution = 1.0m, oversampled = 0.1m, 


In [28]:
lib.reload()

In [30]:
myParameters = MyParameters()
myParameters.Nx = nbAz
myParameters.Ny = nbRg
myParameters.Nover = r_over.size
myParameters.dx = dr_over
myParameters.Naz = Naz
myParameters.Nf = Nf
myParameters.hScene = hScene
myParameters.phi_a_deg = 20 # 20° => 7.4cm

In [31]:
t = time.time()

focusedImg  = np.zeros( (nbAz, nbRg), dtype=complex )
print( "focusedImg.shape = {}".format( focusedImg.shape ) )

#firstRamp = (firstFile+3) * params.rampsPerFile
firstRamp = (firstFile) * params.rampsPerFile
xyz_alt = xyz[firstRamp:, :]

lib.so.backProjectionOmpSlantRange( sceneAz, sceneRg, r_over,
                                   sr.reshape(sr.size),
                                   xyz_alt.reshape(xyz_alt.size), 
                                   focusedImg.reshape(focusedImg.size),
                                   myParameters)

elapsed = time.time() - t
print("execution time = " + str(elapsed))

focusedImg.shape = (1000, 500)
execution time = 528.6352627277374


In [ ]:
min_dB = np.amin( 20 * np.log10(np.abs(focusedImg)) )
max_dB = np.amax( 20 * np.log10(np.abs(focusedImg)) )
med_dB = np.median( 20 * np.log10(np.abs(focusedImg)) )
print("min_dB = {:.2f}, max_dB = {:.2f}, med_dB = {:.2f}".format(min_dB, max_dB, med_dB))
# gps min_dB = -57.20, max_dB = 44.13, med_dB = 4.99
# nav min_dB = -57.88, max_dB = 44.48, med_dB = 2.38

# Plot image

In [ ]:
imgLeft = azMax
imgRight= azMin
imgBottom = rgMax
imgTop = rgMin
extent = [imgLeft, imgRight, imgBottom, imgTop]

## imshow

In [ ]:
cmap = 'gray'

In [ ]:
plt.figure()

#plt.imshow( 20 * np.log10( box_filter( np.abs( imgGroundRange.reshape(nbX, nbY) ), 2 ) ), cmap=cmap )
plt.imshow( 20 * np.log10( np.abs( focusedImg.T ) ), cmap=cmap, extent=extent )
ax = plt.gca()
ax.xaxis.tick_top()
ax.set_aspect("equal")

plt.colorbar()

## imshow box_filter

In [ ]:
plt.figure()

plt.imshow( 20 * np.log10( box_filter( np.abs( focusedImg.T ), 2 ) ), cmap=cmap, extent=extent )
ax = plt.gca()
ax.xaxis.tick_top()
ax.invert_xaxis()

plt.colorbar()

# Build GeoTIFF

**CPLErr GDALDataset::SetGeoTransform 	( 	double *  	padfTransform	)**
Set the affine transformation coefficients. 

**CPLErr GDALDataset::GetGeoTransform 	( 	double *  	padfTransform	)**
Fetch the affine transformation coefficients.

Fetches the coefficients for transforming between pixel/line (P,L) raster space, and projection coordinates (Xp,Yp) space.

Xp = padfTransform[0] + P \* padfTransform[1] + L * padfTransform[2];

Yp = padfTransform[3] + P \* padfTransform[4] + L * padfTransform[5];

In a north up image, padfTransform[1] is the pixel width, and padfTransform[5] is the pixel height. The upper left corner of the upper left pixel is at position (padfTransform[0],padfTransform[3]).

The default transform is (0,1,0,0,0,1) and should be returned even when a CE_Failure error is returned, such as for formats that don't support transformation to projection coordinates.

This method does the same thing as the C GDALGetGeoTransform() function.

In [ ]:
from osgeo import gdal
from osgeo import osr
import json

In [ ]:
driver = gdal.GetDriverByName("GTiff")

In [ ]:
tiffName = data_dir + "/focusedImg.tif"
tiffNameRaw = data_dir + "/focusedImgRaw.tif"
width = nbRg
height = nbAz
dst_ds = driver.Create(tiffName, width, height, 1, gdal.GDT_Float64)
dst_ds_raw = driver.Create(tiffNameRaw, width, height, 1, gdal.GDT_Float64)

In [ ]:
trackFilename = data_dir + "/track_model.json"
with open( trackFilename, 'r' ) as f:
    track_model = json.load( f )

In [ ]:
def sceneToEpsg(x, y, sceneReferencePointXY, trackModel):
    theta = np.arctan2(trackModel["ux"][1], trackModel["ux"][0])
    epsg_x = ( np.cos(theta) * x - np.sin(theta) * y ) + sceneReferencePointXY[0]
    epsg_y = ( np.sin(theta) * x + np.cos(theta) * y ) + sceneReferencePointXY[1]
    return epsg_x, epsg_y

In [ ]:
# Specify raster location through geotransform array
# (upperleftx, scalex, skewx, upperlefty, skewy, scaley)
# Scale = size of one pixel in units of raster projection
# this example below assumes 100x100
sceneReferencePoint = (1365250.4172955733, 7266427.555637825)
if 0:
    upperleftx = 0
    scalex = -d_az
    skewx = 0
    upperlefty = 0
    skewy = 0
    scaley = -d_rg
else:
    theta = np.arctan2(track_model["ux"][1], track_model["ux"][0])
    upperleftx, upperlefty = sceneToEpsg( sceneAz[0], sceneRg[0], sceneReferencePoint, track_model )
    scalex = d_az * (+np.cos(theta))
    skewx = d_az  * (-np.sin(theta))
    skewy = d_rg  * (+np.sin(theta))
    scaley = d_rg * (+np.cos(theta))

gt = [upperleftx, scalex, skewx, upperlefty, skewy, scaley]

# Set location
dst_ds.SetGeoTransform(gt)

In [ ]:
dst_ds.GetRasterBand(1).WriteArray(np.abs(focusedImg))
dst_ds_raw.GetRasterBand(1).WriteArray(np.abs(focusedImg))

In [ ]:
srs = osr.SpatialReference()            # establish encoding
srs.ImportFromEPSG(3948)                # RGF93 / CC48 Projected coordinate system
dst_ds.SetProjection(srs.ExportToWkt()) # export coords to file
dst_ds.FlushCache()                     # write to disk

dst_ds_raw.FlushCache()                     # write to disk

In [ ]:
nbAz, nbRg

# Save focused image

In [ ]:
xStart = sceneX[0][0]
yStart = sceneY[0][0]
xStop = sceneX[-1][-1]
yStop = sceneY[-1][-1]

if groundRange == 1: # [0] ground range, [1] slant range
    if rampUp == 1:
        focusedImageFilename = "/" + data_date \
        + " {} {} GR {:.2f} {:.2f} {} AZ {} {} {} EL {} PHI {} rampUp alt xyz".format( 
            firstFile, lastFile,
            y[0], y[-1], d_y, 
            x[0], x[-1], d_x, 
            hScene,
            myParameters.phi_a_deg)
    else:
        focusedImageFilename = "/" + data_date \
        + " {} {} P0({:.2f},{:.2f}) P1({:.2f},{:.2f}) DX{:.2f} DY{:.2f} EL{} PHI{} rampDown".format( 
            firstFile, lastFile,
            xStart, yStart,
            xStop, yStop, 
            d_x, d_y,
            hScene,
            myParameters.phi_a_deg)
else:
    if rampUp == 1:
        focusedImageFilename = "/" + data_date \
        + " {} {} SR {:.2f} {:.2f} {} AZ {} {} {} EL {} PHI {} rampUp".format( 
            firstFile, lastFile,
            y[0], y[-1], d_y, 
            x[0], x[-1], d_x, 
            hScene,
            myParameters.phi_a_deg)
    else:
        focusedImageFilename = "/" + data_date \
        + " {} {} SR {:.2f} {:.2f} {} AZ {} {} {} EL {} PHI {} rampDown".format( 
            firstFile, lastFile,
            y[0], y[-1], d_y, 
            x[0], x[-1], d_x, 
            hScene,
            myParameters.phi_a_deg)
    
if withHanning:
    focusedImageFilename = focusedImageFilename + "Hann"
if nav:
    focusedImageFilename = focusedImageFilename + "Nav"
else:
    focusedImageFilename = focusedImageFilename + "Gps"
    
print( focusedImageFilename )

# /2018_06_27_12_39_39 0 49 GR 1743.32 1916.78 1.0 AZ 8050.16 6646.63 1.0 EL 50 PHI 20.0 rampDown Hann

In [ ]:
np.save( data_dir + focusedImageFilename, focusedImg )

# Compare images

In [ ]:
name = "2018_06_27_12_39_39 0 49 GR 1743.32 2224.30 1.0 AZ 8050.16 7040.87 1.0 EL 50 PHI 20.0 rampDown Hann nav.npy"
filename = data_dir + "/" + name
img_nav = np.load( filename )

In [ ]:
cmap="gray"

plt.figure()

ax = plt.subplot(121)
plt.imshow( 20 * np.log10( box_filter( np.abs( img_20 ), 2 ) ), cmap=cmap )
plt.grid()
plt.title("20")
plt.colorbar()

plt.subplot(122, sharex=ax, sharey=ax)
plt.imshow( 20 * np.log10( box_filter( np.abs( img_25 ), 2 ) ), cmap=cmap )
plt.grid()
plt.title("25")
plt.colorbar()

In [ ]:
cmap="gray"

plt.figure()

plt.pcolormesh( x, y, 20 * np.log10( box_filter( np.abs( img_20 ), 2 ) ), cmap=cmap )
plt.plot(J1[0], J1[1], 'Dy', markerEdgecolor='k' )
plt.plot(J4[0], J4[1], 'Dy', markerEdgecolor='k' )
plt.plot(J5[0], J5[1], 'Dc', markerEdgecolor='k' )
plt.axes().set_aspect('equal')
plt.grid()
plt.title("20")

In [ ]:
plt.figure()

plt.pcolormesh( x, y, 20 * np.log10( box_filter( np.abs( img_25 ), 2 ) ), cmap=cmap )
plt.plot(J1[0], J1[1], 'Dy', markerEdgecolor='k' )
plt.plot(J4[0], J4[1], 'Dy', markerEdgecolor='k' )
plt.plot(J5[0], J5[1], 'Dc', markerEdgecolor='k' )
plt.axes().set_aspect('equal')
plt.grid()
plt.title("25")

In [ ]:
d_x = 1.
d_y = 1.
nbX = 501
nbY = 501
shiftY = 100
baseLineX = J5[0] + ux[0] * np.arange(nbX)
baseLineY = J5[1] + ux[1] * np.arange(nbX)
sceneX_1 = baseLineX + uy[0] * shiftY
sceneY_1 = baseLineY + uy[1] * shiftY

for n in range(1, nbY):
    newX = baseLineX + uy[0] * (n + shiftY)
    newY = baseLineY + uy[1] * (n + shiftY)
    sceneX_1 = np.concatenate((sceneX_1, newX))
    sceneY_1 = np.concatenate((sceneY_1, newY))

In [ ]:
plt.figure()

ax = plt.subplot(121)
x = sceneX_1.reshape(nbY, nbX)
y = sceneY_1.reshape(nbY, nbX)
plt.pcolormesh( x, y, 20 * np.log10( box_filter( np.abs( img_20 ), 2 ) ), cmap=cmap )
plt.plot(J1[0], J1[1], 'Dy', markerEdgecolor='k' )
plt.plot(J4[0], J4[1], 'Dy', markerEdgecolor='k' )
plt.plot(J5[0], J5[1], 'Dc', markerEdgecolor='k' )
plt.gca().set_aspect('equal')
plt.grid()
plt.title("20")

plt.subplot(122, sharex=ax, sharey=ax)
x = sceneX.reshape(401, 401)
y = sceneY.reshape(401, 401)
plt.pcolormesh( x, y, 20 * np.log10( box_filter( np.abs( img_20_05 ), 2 ) ), cmap=cmap )
plt.plot(J1[0], J1[1], 'Dy', markerEdgecolor='k' )
plt.plot(J4[0], J4[1], 'Dy', markerEdgecolor='k' )
plt.plot(J5[0], J5[1], 'Dc', markerEdgecolor='k' )
plt.grid()
plt.title("25")

# Save image as png

In [ ]:
plt.imsave( data_dir + focusedImageFilename + ".png",
           20 * np.log10( box_filter( np.abs( np.flip( imgGroundRange.T, 1 ) ), 5 ) ), 
           cmap="gray" )

# WGS84 to ECEF

In [ ]:
# LLA2ECEF - convert latitude, longitude, and altitude to
#            earth-centered, earth-fixed (ECEF) cartesian
# 
# USAGE:
# [x,y,z] = lla2ecef(lat,lon,alt)
# 
# x = ECEF X-coordinate (m)
# y = ECEF Y-coordinate (m)
# z = ECEF Z-coordinate (m)
# lat = geodetic latitude (radians)
# lon = longitude (radians)
# alt = height above WGS84 ellipsoid (m)
# 
# Notes: This function assumes the WGS84 model.
#        Latitude is customary geodetic (not geocentric).
# 
# Source: "Department of Defense World Geodetic System 1984"
#         Page 4-4
#         National Imagery and Mapping Agency
#         Last updated June, 2004
#         NIMA TR8350.2
# 
# Michael Kleder, July 2005

def lla2ecef(lat,lon,alt):

    # WGS84 ellipsoid constants:
    a = 6378137 # semi-major axis
    e = 8.1819190842622e-2 # First Eccentricity

    lat = lat * np.pi / 180
    lon = lon * np.pi / 180
    
    # intermediate calculation
    # (prime vertical radius of curvature)
    N = a / np.sqrt(1 - e**2 * np.sin(lat)**2)

    # results:
    x = (N+alt) * np.cos(lat) * np.cos(lon)
    y = (N+alt) * np.cos(lat) * np.sin(lon)
    z = ((1-e**2) * N + alt) * np.sin(lat)

    return (x, y, z)

In [ ]:
disc_ecef = lla2ecef(disc[0],disc[1],discElevation)
threePools_ecef = lla2ecef(threePools[0],threePools[1],threePoolsElevation)

In [ ]:
def distance(a,b):
    dx = (a[0]-b[0])**2
    dy = (a[1]-b[1])**2
    dz = (a[2]-b[2])**2
    return (dx+dy+dz)**0.5

In [ ]:
distance(disc_ecef, threePools_ecef)

In [ ]:
distance( (disc_epsg[0], disc_epsg[1], discElevation),
         (threePools_epsg[0], threePools_epsg[1], threePoolsElevation) )

In [ ]:
antenna = [48.512965 , -1.538558]
antenna_epsg = wgs84ToEpsg( antenna[::-1], epsg3948, shiftXY, origXY )
antenna_elevation = 300
antenna_ecef = lla2ecef(antenna[0],antenna[1],antenna_elevation)

In [ ]:
distance(threePools_ecef, antenna_ecef)

In [ ]:
distance( (antenna_epsg[0], antenna_epsg[1], antenna_elevation),
         (threePools_epsg[0], threePools_epsg[1], threePoolsElevation) )